In [ ]:
type Action =
| WalkTo of string
| Open of string

type Graph = { 
    nodes: Map<string, Action seq>
    flow: Map<string,int>
}
module Graph =
    module Parser =
        open System.Text.RegularExpressions

        let regex = Regex(@"^Valve (?<valve>\w.) has flow rate=(?<flow>\d+); tunnels? leads? to valves? (?<valves>(?:\w.)(:?,\s\w.)*)$")

        let folder graph line : Graph =
            let m = regex.Match(line)

            let label = m.Groups.["valve"].Value
            let flow = m.Groups.["flow"].Value |> int
            let valves = 
                m.Groups.["valves"].Value.Split(", ")
                |> Seq.map Action.WalkTo
                |> Seq.append [ Action.Open label ]

            { graph with 
                nodes = graph.nodes |> Map.add label valves
                flow = graph.flow |> Map.add label flow
            }

        let parse lines =
            lines |> Seq.fold folder { nodes = Map.empty; flow = Map.empty }

    let flow graph action =
        let label = match action with WalkTo w -> w | Open o -> o
        graph.flow |> Map.find label

    let neighbours graph action =
        let label = match action with WalkTo w -> w | Open o -> o
        graph.nodes 
        |> Map.find label
    

In [ ]:
let ResolutionFolder = __SOURCE_DIRECTORY__
let graph =
    File.ReadLines( ResolutionFolder + "/testcase16.txt")
    |> Graph.Parser.parse

In [7]:
type AStar<'T when 'T :comparison> = {
    goal: 'T
    current: 'T
    openSet: Set<'T>
    gScore: Map<'T, int>
    fScore: Map<'T, int>
    cameFrom: Map<'T,'T>
} with
    member this.Next() =
        match this.openSet |> List.ofSeq with
        | [] -> this
        | head :: tail ->
            { this with
                current = head
                openSet = tail |> Set.ofList
            }

type ScoreFun<'T> = 'T -> 'T -> int
type NeighbourFun<'T> = 'T -> 'T seq
module AStar =
    let create start goal (scorer:ScoreFun<_>) (neighbours:NeighbourFun<_>) : AStar<_> =
        {
            goal = goal
            current = start
            openSet = Set.empty
            gScore = [(start,0)] |> Map.ofList
            fScore = [(start, scorer start goal)] |> Map.ofList
            cameFrom = Map.empty
        }

    let next state = 
        match state.openSet |> List.ofSeq with
        | [] -> state
        | head :: tail ->
            { state with 
                current = head
                openSet = tail |> Set.ofList
            }

    let rec find (score:ScoreFun<_>) (neighbours:NeighbourFun<_>) state =
        if state.goal = state.current then state else
        let state = next state

        let folder state node =
            let tentative = (state.gScore.Item (state.current)) + + (score state.goal node)
            let actual = state.gScore.TryFind node |> Option.defaultValue 99999
            if tentative < actual then
                state
            else
                state
        
        state.current
        |> neighbours
        |> Seq.fold folder state
        |> find score neighbours

In [ ]:
type Simulation = {
    graph: Graph
    current: Action
    released: int
    steps: int
    closed: Action seq
    opened: Action seq
} with 
    member this.FlowRate () =
        this.opened
        |> Seq.map (Graph.flow this.graph)
        |> Seq.sum

module Simulation =
    let rec step acc state =